<a href="https://colab.research.google.com/github/GenaroHacker/actionable/blob/main/database_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title sql_core
import sqlite3

def create_table( database_name , table_name , table_structure ):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    try:
        my_cursor.execute('''
            CREATE TABLE {} ( {} )
        '''.format(table_name, table_structure))
        my_connection.commit()
        my_connection.close()
        #Table created successfully
    except sqlite3.OperationalError:
        #Table already exists
        pass

def insert_record(database_name, record):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(record)
    my_connection.commit()
    my_connection.close()

def insert_several_records(database_name, multiple_records):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    for i in multiple_records:
        my_cursor.execute(i)
    my_connection.commit()
    my_connection.close()

def read_records(database_name, table_name):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute("SELECT * FROM {}".format(table_name))
    records=my_cursor.fetchall()
    my_connection.close()
    return records

def read_last_record(database_name, table_name):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute("SELECT * FROM {} ORDER BY ID DESC LIMIT 1".format(table_name))
    records=my_cursor.fetchall()
    my_connection.close()
    return records[0]

def update_record(database_name, record):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(record)
    my_connection.commit()
    my_connection.close()

def remove_record(database_name, record):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(record)
    my_connection.commit()
    my_connection.close()

def run_command(database_name, command):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(command)
    my_connection.commit()
    my_connection.close()

if __name__ == "__main__":
    #Create one Table
    columns = """
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            ITEM_NAME VARCHAR(50),
            PRICE INTEGER,
            SECTION VARCHAR(20)"""
    create_table("BaseProducts","TableProducts",columns)

    #Insert one record
    insert_record("BaseProducts","INSERT INTO TableProducts VALUES (NULL,'BALL',10,'SPORT')")
    #Record inserted successfully!

    #Insert several records
    insert_several_records("BaseProducts",[
        "INSERT INTO TableProducts VALUES (NULL,'GOLF STICK',25,'SPORT')",
        "INSERT INTO TableProducts VALUES (NULL,'GLASS',20,'CERAMIC')",
        "INSERT INTO TableProducts VALUES (NULL,'T-SHIRT',5,'CLOTHES')"
    ])
    #Records inserted successfully!

    #Read all records
    list_of_tuples = read_records("BaseProducts","TableProducts")

    #Read last record
    last_record = read_last_record("BaseProducts","TableProducts")

    #Update record
    update_record("BaseProducts","UPDATE TableProducts SET ITEM_NAME='BALL NAME UPDATED' WHERE ID=3")
    #The record with ID=3 has been updated successfully!

    #Remove record
    remove_record("BaseProducts","DELETE FROM TableProducts WHERE ID=3")
    #The record with ID=4 has been removed successfully!

In [2]:
# @title set up
import sqlite3
from google.colab import files
import os

# Constant for the database name
DATABASE_NAME = 'TaskManagement.db'

def create_table(database_name, table_name, table_structure):
    my_connection = sqlite3.connect(database_name)
    my_cursor = my_connection.cursor()
    try:
        my_cursor.execute(f'CREATE TABLE {table_name} ({table_structure})')
        my_connection.commit()
    except sqlite3.OperationalError:
        pass
    finally:
        my_connection.close()

def insert_record(database_name, record):
    my_connection = sqlite3.connect(database_name)
    my_cursor = my_connection.cursor()
    my_cursor.execute(record)
    my_connection.commit()
    my_connection.close()

def read_last_record(database_name, table_name):
    my_connection = sqlite3.connect(database_name)
    my_cursor = my_connection.cursor()
    my_cursor.execute(f"SELECT * FROM {table_name} ORDER BY ID DESC LIMIT 1")
    records = my_cursor.fetchall()
    my_connection.close()
    return records[0]

def check_record_exists(table_name, column_name, value):
    my_connection = sqlite3.connect(DATABASE_NAME)
    my_cursor = my_connection.cursor()
    my_cursor.execute(f"SELECT ID FROM {table_name} WHERE {column_name} = ?", (value,))
    records = my_cursor.fetchone()
    my_connection.close()
    return records[0] if records else None

def initialize_database():
    user_choice = input("Choose an option:\n1. Create empty database\n2. Create database with example data\n3. Upload database\n")
    if user_choice == '1':
        create_empty_database()
    elif user_choice == '2':
        create_example_database()
    elif user_choice == '3':
        uploaded_files = files.upload()
        for filename in uploaded_files.keys():
            print(f"Uploaded file: {filename}")
            # Rename the uploaded file to match the DATABASE_NAME
            if filename != DATABASE_NAME:
                os.rename(filename, DATABASE_NAME)
                print(f"Renamed '{filename}' to '{DATABASE_NAME}'")
    else:
        print("Invalid choice. Please select a valid option.")

def create_empty_database():
    tables = {
        "PLACES": "ID INTEGER PRIMARY KEY, NAME TEXT UNIQUE, RANK INTEGER",
        "CONTEXTS": "ID INTEGER PRIMARY KEY, PLACE_ID INTEGER, NAME TEXT UNIQUE, RANK INTEGER",
        "TAGS": "ID INTEGER PRIMARY KEY, CONTEXT_ID INTEGER, NAME TEXT UNIQUE, RANK INTEGER",
        "TASKS": "ID INTEGER PRIMARY KEY, TAG_ID INTEGER, NAME TEXT UNIQUE, RANK INTEGER",
        "STEPS": "ID INTEGER PRIMARY KEY, TASK_ID INTEGER, NAME TEXT, ORDER_SEQUENCE INTEGER, MINUTES INTEGER, UNIQUE(TASK_ID, ORDER_SEQUENCE)",
        "PLACEHOLDERS": "ID INTEGER PRIMARY KEY, TYPE TEXT, VALUE TEXT, RANK INTEGER",
        "HISTORY": "ID INTEGER PRIMARY KEY, STEP_ID INTEGER, ACTION TEXT, TIME TEXT"
    }
    for table_name, table_structure in tables.items():
        create_table(DATABASE_NAME, table_name, table_structure)

def create_example_database():
    create_empty_database()
    # Add code to insert example data into each table

def check_or_insert(table, key_column, key_value, additional_fields=None):
    record_id = check_record_exists(table, key_column, key_value)
    if record_id is None:
        fields = f"{key_column}, " + ", ".join(additional_fields.keys()) if additional_fields else key_column
        values = f"'{key_value}', " + ", ".join(str(v) for v in additional_fields.values()) if additional_fields else f"'{key_value}'"
        insert_record(DATABASE_NAME, f"INSERT INTO {table} ({fields}) VALUES ({values})")
        return read_last_record(DATABASE_NAME, table)[0]
    return record_id

def add_task(task_input):
    place = task_input["place"]
    context = task_input["context"]
    tag = task_input["tag"]
    task_name = task_input["task"]
    steps = task_input["steps"]

    place_id = check_or_insert("PLACES", "NAME", place)
    context_id = check_or_insert("CONTEXTS", "NAME", context, additional_fields={"PLACE_ID": place_id})
    tag_id = check_or_insert("TAGS", "NAME", tag, additional_fields={"CONTEXT_ID": context_id})
    task_id = check_or_insert("TASKS", "NAME", task_name, additional_fields={"TAG_ID": tag_id})

    for index, step in enumerate(steps, start=1):
        if not check_step_exists(task_id, index):
            insert_record(DATABASE_NAME, f"INSERT INTO STEPS (TASK_ID, NAME, ORDER_SEQUENCE, MINUTES) VALUES ({task_id}, '{step}', {index}, NULL)")

def check_or_insert(table, key_column, key_value, additional_fields=None):
    record_id = check_record_exists(table, key_column, key_value)
    if record_id is None:
        fields = f"{key_column}, " + ", ".join(additional_fields.keys()) if additional_fields else key_column
        values = f"'{key_value}', " + ", ".join(str(v) for v in additional_fields.values()) if additional_fields else f"'{key_value}'"
        insert_record(DATABASE_NAME, f"INSERT INTO {table} ({fields}) VALUES ({values})")
        return read_last_record(DATABASE_NAME, table)[0]
    return record_id

def check_step_exists(task_id, order_sequence):
    my_connection = sqlite3.connect(DATABASE_NAME)
    my_cursor = my_connection.cursor()
    my_cursor.execute("SELECT COUNT(*) FROM STEPS WHERE TASK_ID = ? AND ORDER_SEQUENCE = ?", (task_id, order_sequence))
    count = my_cursor.fetchone()[0]
    my_connection.close()
    return count > 0

def check_record_exists(table_name, column_name, value):
    my_connection = sqlite3.connect(DATABASE_NAME)
    my_cursor = my_connection.cursor()
    my_cursor.execute(f"SELECT ID FROM {table_name} WHERE {column_name} = ?", (value,))
    records = my_cursor.fetchone()
    my_connection.close()
    return records[0] if records else None

def download_database():
    files.download(DATABASE_NAME)

def add_placeholders(placeholders_dict):
    for placeholder_type, values in placeholders_dict.items():
        for value in values:
            if not check_placeholder_exists(placeholder_type, value):
                insert_record(DATABASE_NAME, f"INSERT INTO PLACEHOLDERS (TYPE, VALUE) VALUES ('{placeholder_type}', '{value}')")

def check_placeholder_exists(placeholder_type, value):
    my_connection = sqlite3.connect(DATABASE_NAME)
    my_cursor = my_connection.cursor()
    my_cursor.execute("SELECT COUNT(*) FROM PLACEHOLDERS WHERE TYPE = ? AND VALUE = ?", (placeholder_type, value))
    count = my_cursor.fetchone()[0]
    my_connection.close()
    return count > 0


In [5]:
# @title Main Execution

# Main Execution
if __name__ == "__main__":
    initialize_database()

    # Example task input
    task = {
        "place": "home",
        "context": "create",
        "tag": "guitar",
        "task": "loop %note%",
        "steps": [
            "identify the note in each string",
            "start the metronome at 30",
            "loop over each string 3 times"
        ]
    }

    add_task(task)

    # Example usage of add_placeholders
    placeholders = {
        "note": ["C", "C#", "D"],
        "color": ["red", "blue", "green"]
    }

    add_placeholders(placeholders)

    download_database()

Choose an option:
1. Create empty database
2. Create database with example data
3. Upload database
3


Saving TaskManagement (1).db to TaskManagement (1).db
Uploaded file: TaskManagement (1).db
Renamed 'TaskManagement (1).db' to 'TaskManagement.db'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>